<a href="https://colab.research.google.com/github/dianakang/DIANA_TimeSeries/blob/master/%5B%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9C%BC%EB%A1%9C_%EA%B1%B7%EB%8A%94_%EC%8B%9C%EA%B3%84%EC%97%B4_%EC%98%88%EC%B8%A1%5D_Ch5_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목차

*   5.1. concatenate 메서드 - 여러 모델 합치기
*   5.2. Merge Layer
  *   5.2.1. Add()
  *   5.2.2. Subtract()
  *   5.2.3. Multiply()
  *   5.2.4. Average()
  *   5.2.5. Maximum()
  *   5.2.6. Minimun()
  *   5.2.7. Concatenate()
  *   5.2.8. Dot()
  *   5.2.9. add()
  *   5.2.10. subtract()
  *   5.2.11. multiply()
  *   5.2.12. average()
  *   5.2.13. maximum()
  *   5.2.14. minimum()
  *   5.2.15. concatenate()
  *   5.2.16. dot()



---

# 5. 앙상블

## 5.1. concatenate 메서드 - 여러 모델 합치기

이번 5장에서는 여러개의 모델을 합치는 방법을 배울 것이다. 각각 300개의 데이터를 가지고 있는 2개의 x와 100개의 데이터를 가지고 있는 1개의 y를 준비한다.

In [10]:
# 1. 데이터 준비하기
import numpy as np

x1 = np.array([range(100), range(311,411), range(100)])
x2 = np.array([range(101,201), range(311,411), range(101,201)])

y = np.array([range(501,601)])

# 행열 변환 - 현재의 shape은 (3,100)이므로 (100,3)으로 reshape 해야한다.
x1 = np.transpose(x1)
y = np.transpose(y)
x2 = np.transpose(x2)

# train, test, validation 분리
from sklearn.model_selection import train_test_split
x1_train, x1_test, y_train, y_test = train_test_split(x1, y, random_state=66, test_size=0.4, shuffle=False)
x1_val, x1_test, y_val, y_test = train_test_split(x1_test, y_test, random_state=66, test_size=0.5, shuffle=False)

x2_train, x2_test = train_test_split(x2, random_state=66, test_size=0.4, shuffle=False)
x2_val, x2_test = train_test_split(x2_test, random_state=66, test_size=0.5, shuffle=False)


# 2. 모델 구성하기 - 앙상블 모델
from keras.models import Sequential, Model
from keras.layers import Dense, Input

## Input 레이어를 이용해 레이어 만들기
input1 = Input(shape=(3,))  ## 함수형 모델로 input1 만들기
dense1 = Dense(100, activation='relu')(input1)
dense1_2 = Dense(30)(dense1)
dense1_3 = Dense(7)(dense1_2)

input2 = Input(shape=(3,))
dense2 = Dense(50, activation='relu')(input2)
dense2_2 = Dense(7)(dense2)

# input1과 input2를 merge하기 -> concatenate 함수 이용
from keras.layers.merge import concatenate
merge1 = concatenate([dense1_3, dense2_2])   ## concatenate 함수에 매개변수로 상단 2개의 모델 최하위 레이어의 이름, 리스트 방식([])으로 입력

# Concatenate 클래스 이용 -> 사용법은 concatenate와 거의 유사하며 ()를 더 붙여주면 된다.
from keras.layers.merge import Concatenate
merge1 = Concatenate()([dense1_3, dense2_2])

# 레이어 엮어주기
model1 = Dense(10)(merge1)
model2 = Dense(5)(model1)
output = Dense(1)(model2)

# 모델 정의하기 - Model() 사용
model = Model(inputs = [input1, input2], outputs = output)
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense_21 (Dense)                (None, 100)          400         input_7[0][0]                    
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 30)           3030        dense_21[0][0]                   
____________________________________________________________________________________________

-> 다섯 번째 dense와 여섯 번째 dense 사이에서 conatenate가 잘 이루어진 것을 확인할 수 있다.

In [13]:
# 3. 모델 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit([x1_train, x2_train],
          y_train, epochs=300, batch_size=1,
validation_data=([x1_val, x2_val] , y_val))

Epoch 1/300
60/60 [==============================] - 1s 5ms/step - loss: 151.0917 - mse: 151.0917 - val_loss: 13.3818 - val_mse: 13.3818
Epoch 2/300
60/60 [==============================] - 0s 2ms/step - loss: 0.7025 - mse: 0.7025 - val_loss: 13.1193 - val_mse: 13.1193
Epoch 3/300
60/60 [==============================] - 0s 3ms/step - loss: 0.0625 - mse: 0.0625 - val_loss: 12.3252 - val_mse: 12.3252
Epoch 4/300
60/60 [==============================] - 0s 3ms/step - loss: 0.0326 - mse: 0.0326 - val_loss: 9.0344 - val_mse: 9.0344
Epoch 5/300
60/60 [==============================] - 0s 2ms/step - loss: 0.2688 - mse: 0.2688 - val_loss: 17.4953 - val_mse: 17.4953
Epoch 6/300
60/60 [==============================] - 0s 2ms/step - loss: 0.6402 - mse: 0.6402 - val_loss: 13.8744 - val_mse: 13.8744
Epoch 7/300
60/60 [==============================] - 0s 2ms/step - loss: 0.1286 - mse: 0.1286 - val_loss: 12.2093 - val_mse: 12.2093
Epoch 8/300
60/60 [==============================] - 0s 3ms/step - 

-> 훈련을 할 때 x의 입력 데이터가 2개이므로 list 형태를 취해야한다.

  [x1_train, x2_train] 그리고  Validation 역시 list 형태로 입력하는 것을 확인해야 한다. 이제 평가 예측을 하고 모델을 판단해보자.

In [14]:
# 4. 평가 예측
mse = model.evaluate([x1_test, x2_test], y_test, batch_size=1)  ## 모델 평가
print("mse : ", mse)

y_predict = model.predict([x1_test, x2_test])  ## 평가 시에도 마찬가지로 x의 입력 데이터가 2개이므로 list 형태를 취했다.

for i in range(len(y_predict)):
  print(y_test[i], y_predict[i])

20/20 [==============================] - 0s 1ms/step - loss: 11.5143 - mse: 11.5143
mse :  [11.514313697814941, 11.514313697814941]
[581] [582.6362]
[582] [583.78735]
[583] [584.93585]
[584] [586.08435]
[585] [587.233]
[586] [588.3814]
[587] [589.5337]
[588] [590.6893]
[589] [591.84467]
[590] [593.00006]
[591] [594.1532]
[592] [595.3055]
[593] [596.4578]
[594] [597.6751]
[595] [598.9414]
[596] [600.2076]
[597] [601.47394]
[598] [602.7402]
[599] [604.0066]
[600] [605.2728]


## 5.2. Merge Layer

concatenate은 단순히 모델 2개를 엮어주는 역할을 한다. 다시 말하면 노드들이 단순하게 더하기 형태로 합쳐지는 것이다.

concatenate()함수 외에도 다양한 merge layer들이 있는데, 이를 살펴보자!

### 5.2.1. Add()




*   **keras.layers.Add()**

: layers끼리의 단순한 더하기이다. 동일한 모양의 텐서 목록을 입력으로 사용하고 하나의 텐서(같은 모양)를 반환한다.


In [15]:
# Add() 함수
import keras

input1 = keras.layers.Input(shape=(16,))
x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(32,))
x2 = keras.layers.Dense(8, activation='relu')(input2)

added = keras.layers.Add()([x1, x2])

out = keras.layers.Dense(4)(added)
model = keras.models.Model(inputs=[input1, input2], outputs=out)

### 5.2.2. Subtract()

*   **keras.layers.Subtract()**

: layers끼리의 단순한 뺄셈이다. 동일한 모양의 크기 2인 텐서 목록을 입력으로 사용하고, 동일한 모양의 단일 텐서(입력[0] - 입력[1])을 반환한다.

In [16]:
# Subtract() 함수
import keras

input1 = keras.layers.Input(shape=(16,))
x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(32,))
x2 = keras.layers.Dense(8, activation='relu')(input2)

subtracted = keras.layers.Subtract()([x1, x2])

out = keras.layers.Dense(4)(subtracted)
model = keras.models.Model(inputs=[input1, input2], outputs=out)

### 5.2.3. Multiply()

*   **keras.layers.Multiply()**

: layers끼리의 입력 목록을 (요소 별로)곱하는 레이어이다. 동일한 모양의 텐서 목록을 입력으로 사용하고 하나의 텐서를 반환한다.

### 5.2.4. Average()

*   **keras.layers.Average()**

: 인풋 리스트의 평균을 내는 레이어이다. 동일한 모양의 텐서 목록을 입력으로 사용하고 하나의 텐서를 반환한다.

### 5.2.5. Maximum()

*   **keras.layers.Maximum()**

: 입력 목록의 최대(요소 별로)를 계산하는 레이어이다. 동일한 모양의 텐서 목록을 입력으로 사용하고 하나의 텐서를 반환한다.

### 5.2.6. Minimum()

*   **keras.layers.Minimum()**

: 입력 목록의 최소(요소 별로)를 계산하는 레이어이다. 동일한 모양의 텐서 목록을 입력으로 사용하고 하나의 텐서를 반환한다.

### 5.2.7. Concatenate()

*   **keras.layers.Concatenate(axis=-1)**

: 입력 목록을 연결하는 계층이다. 연결 축을 제외한 동일한 모양의 텐서 목록을 입력으로 사용하고 모든 입력의 연결인 단일 텐서를 반환한다. 

소문자 concatenate과 사용할 때 비교하면서 사용해야 한다. 디폴트는 axis=-1이다.

### 5.2.8. Dot()

*   **keras.layers.Dot(axes, normalize=False)**

: 두 텐서에서 샘플 간 내적을 계산하는 레이어이다. 2개의 텐서 a와 b의 모양(batch_size, n)목록에 적용되는 경우 출력은 모양의 텐서(batch_size,1)가 된다. 

여기서 각 항목 i는 a [i]와 b [j] 사이의 내적이다. 

### 5.2.9. add()

*   **keras.layers.add(inputs)**

: Add와 같은 결과를 나타내지만, 함수형 add는 사용법이 약간 다르다.  

In [17]:
# add() 함수
import keras

input1 = keras.layers.Input(shape=(16,))
x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(32,))
x2 = keras.layers.Dense(8, activation='relu')(input2)

added = keras.layers.add([x1, x2])

out = keras.layers.Dense(4)(added)
model = keras.models.Model(inputs=[input1, input2], outputs=out)



  *   Add의 경우

  : added = keras.layers.Add()([x1, x2]) 이렇게 사용하지만,

  *   add의 경우는 

  : added = keras.layers.add([x1, x2]) 이렇게 사용한다.

  



### 5.2.10. subtract()

*   **keras.layers.subtract(inputs)**

: Subtract와 동일한 사용법이고 ()를 주의하면 된다. 레이어 간 노드의 빼기이다. 

In [18]:
# subtract() 함수
import keras

input1 = keras.layers.Input(shape=(16,))
x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(32,))
x2 = keras.layers.Dense(8, activation='relu')(input2)

subtracted = keras.layers.subtract([x1, x2])

out = keras.layers.Dense(4)(subtracted)
model = keras.models.Model(inputs=[input1, input2], outputs=out)

### 5.2.11. multiply()

*   **keras.layers.multiply(inputs)**

: Multiply와 동일한 사용법이고 ()를 주의하면 된다. Input tensor가 최소 2개 이상이다.

### 5.2.12. average()

*   **keras.layers.average(inputs)**

: Average와 동일한 사용법이고 ()를 주의하면 된다. Input tensor가 최소 2개 이상이다.

### 5.2.13. maximum()

*   **keras.layers.maximum(inputs)**

: Maximum과 동일한 사용법이고 ()를 주의하면 된다. Input tensor가 최소 2개 이상이다. 입력한 텐서 중 최소값 텐서를 리턴한다.

### 5.2.14. minimum()

*   **keras.layers.minimum(inputs)**

: Minimum과 동일한 사용법이고 ()를 주의하면 된다. Input tensor가 최소 2개 이상이다. 입력한 텐서 중 최소값 텐서를 리턴한다.

### 5.2.15. concatenate()

*   **keras.layers.concatenate(inputs, axis=-1)**

: Concatenate과 비교하여 잘 사용하면 되고, 디폴트 axis는 -1이다.

### 5.2.16. dot()

*   **keras.layers.dot(inputs, axes, normalize=False)**

: inputs는 2개 이상의 텐서이다. normalize()는 L2 정규화의 여부이다. 디폴트는 False이다. Dot과 동일한 방법이고 ()를 주의하여 사용하면 된다.